In [3]:
#functie care cauta in URL cuvinte cheie care pot contine nume de produs 

import pandas as pd

def process_good(word):
    keywords = ['product', 'item', 'sku', 'brand', 'model', 'name', 'slug', 'tags', 'description', 'code', 'category', 'categ']

    if any(keyword in word for keyword in keywords):
        return True
    return False


In [4]:
#pastreaza URL urile care nu contin aceste cuvinte cheie
import pandas as pd

# Function to check if a word contains only numbers
def contains_only_numbers(word):
    return word.isdigit()

# Function to process the DataFrame and filter out URLs
def process_bad(word):
    keywords = [
        "about",
        "contact",
        "info",
        "information",
        "blog",
        "news",
        "team",
        "services",
        "careers",
        "faq",
        "support",
        "resources",
        "events",
        "gallery",
        "testimonials",
        "privacy-policy",
        "terms-of-service",
        "tos",
        "press",
        "partners",
        "solutions",
        "case-studies",
        "portfolio",
        "our-work",
        "clients",
        "awards",
        "leadership",
        "mission",
        "vision",
        "projects",
        "subscriptions",
        "volunteer",
        "meet-the-team",
        "pricing",
        "productSort",
        "contact-us"
    ]

    if any(keyword in word for keyword in keywords) or contains_only_numbers(word):
        return True
    return False

In [ ]:
df = pd.read_csv("url_tokens.csv")

In [7]:

def prelucrate(received_url):
    if process_good(received_url):
        return 1
    if process_bad(received_url):
        return 0
    
    return 2

In [22]:
!pip3 install urlparse

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from urllib.parse import urlparse, parse_qs
import time

In [24]:
def get_prediction(tokenizer, model, text):
	encoding = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=32)
	outputs = model(**encoding)
	logits = outputs.logits
	probs = torch.nn.functional.softmax(logits, dim=-1)
	label = torch.argmax(probs).item()
	return label

In [25]:
def custom_tokenize_url(url):
    tokens = []
    parsed_url = urlparse(url)
    
    # Tokenize the domain
    path_tokens = parsed_url.path.split('/')
    path_tokens = path_tokens[1:]
    tokens.extend(filter(None, path_tokens))
    # Tokenize the query string
    query_tokens = []
    query_params = parse_qs(parsed_url.query)
    for param, values in query_params.items():
        query_tokens.append(param)
        query_tokens.extend(values)
    tokens.extend(query_tokens)
    
    # Remove empty tokens and return
    return ' '.join(filter(None, tokens))

In [26]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('/home/vlad/Documents/HACK/HBN/URL-Product-Extraction-ML/checkpoint-150500')


In [34]:
from sentence_transformers import SentenceTransformer, util
model_category_classifier = SentenceTransformer('all-mpnet-base-v2')

with open('categorii.txt', 'r') as f:
    categories = f.readlines()
    categories = [cat.strip() for cat in categories]

categories = [cat.lower().strip() for cat in categories]

In [35]:

categ = model_category_classifier.encode(categories)
def assign_category(word):
    w = model_category_classifier.encode(word)

    cosine_scores = util.pytorch_cos_sim(w, categ)
    best_cat_index = cosine_scores.argmax()
    best_cat = categories[best_cat_index]

    return best_cat


In [55]:
import csv

def extract_urls_from_csv(file_path):
    urls = []
    i = 0
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            url = row['url']
            urls.append(url)
            i += 1
    return urls

file_path = '/Users/dragosmac/_workspace_/hbn/url_product_extraction_input_dataset.csv'  # Replace 'your_file.csv' with the path to your CSV file
urls = extract_urls_from_csv(file_path)


In [56]:
len(urls)

321245

In [57]:
for url in urls:
    res = prelucrate(url)
    if res == 1:
        last = url.split('/')[-1]
        if last == '':
            last = url.split('/')[-2]
        with open('good_urls.csv', 'a') as f:
            f.write(f'{url}, {last}, {assign_category(last)}\n')
    elif res == 0:
        continue
    elif res == 2:
        res = get_prediction(tokenizer, model, custom_tokenize_url(url))
        if res == 1:
            last = url.split('/')[-1]
            if last == '':
                last = url.split('/')[-2]
            with open('good_urls.csv', 'a') as f:
                f.write(f'{url}, {last}, {assign_category(last)}\n')
        else:
            continue

In [45]:
urls = urls[0:10000]
good_urls = []
bad_urls = []
still_processing_urls = []
for url in urls:
    process_res = prelucrate(url)
    if process_res == 1:
        good_urls.append(url)
        continue
    if process_res == 0:
        bad_urls.append(url)
        continue
    still_processing_urls.append(url)

for url in still_processing_urls:
    process_res = get_prediction(tokenizer, model, custom_tokenize_url(url))
    if process_res == 1:
        good_urls.append(url)
        continue
    bad_urls.append(url)

full_class = []
names = []
for url in good_urls:
    slashes = url.split('/')
    product = slashes[:-1]
    names.append(product)
    full_class.append([url, product])
    with open("good_url.txt", "a") as file:
        file.write(url + " " + product + "\n")

for name in full_class:
    category = name[1]
    category = assign_category([name[1]])
    name.append(category)

for url in good_urls:
    with open("good_url.txt", "a") as file:
        file.write(url + '\n')

for url in bad_urls:
    with open("bad_url.txt", "a") as file:
        file.write(url + '\n')


TypeError: can only concatenate str (not "list") to str

In [10]:
!pip3 install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.5 MB/s eta 0:00:001.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.2 MB/s eta 0:00:0016.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 25.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 30.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.1 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 30.9 MB/s eta 0:00:0031m33.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 29.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.2 MB